In [521]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service as FirefoxService
from unidecode import unidecode
from pathlib import Path
from datetime import datetime, timedelta
from lingua import Language, LanguageDetectorBuilder
import pandas as pd
import json
import time

In [2]:
365 * 7

2555

In [522]:
# url = "https://www.google.com/travel/hotels/entity/ChYIqtL21OvSv65QGgovbS8wdnB3cTRzEAE/reviews?utm_campaign=sharing&utm_medium=link&utm_source=htls&ts=CAEaIAoCGgASGhIUCgcI6A8QAxgLEgcI6A8QAxgMGAEyAhAAKgkKBToDTUdBGgA&ved=0CAAQ5JsGahcKEwiY1evEleyEAxUAAAAAHQAAAAAQAw"
# url = "https://www.google.com/travel/hotels/entity/ChYIqtL21OvSv65QGgovbS8wdnB3cTRzEAE/reviews?utm_campaign=sharing&utm_medium=link&utm_source=htls"
url = "https://www.google.com/search?sca_esv=0ac22e5c01a3af73&tbm=lcl&sxsrf=ACQVn09l9CR4PluJJcjCU2I3xMwJBUhv3g:1711641232717&q=Agence+CENTURY+21+Alp+Immo+Confiance+Aix-les-Bains+Avis&rflfq=1&num=20&stick=H4sIAAAAAAAAAONgkxIxNDIyMzE3MLU0NjOwMDawMDQ0Md7AyPiK0dwxPTUvOVXB2dUvJDQoUsHIUMExp0DBMzc3X8E5Py8tMxEk65hZoZuTWqzrlJiZV6zgWJZZvIiVXJ0ACrRipY8AAAA&rldimm=12264705936083081143&hl=fr-FR&sa=X&ved=2ahUKEwjRqaK9qJeFAxXiV6QEHeGXCMEQ9fQKegQINxAF&biw=1920&bih=927&dpr=1#lkt=LocalPoiReviews"

In [523]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('disable-translate')
chrome_options.add_argument('--lang=es')

In [524]:
# driver = webdriver.Chrome(options=chrome_options)
firefox_options = webdriver.FirefoxOptions()
firefox_options.add_argument('--disable-gpu')
firefox_options.add_argument('--ignore-certificate-errors')
firefox_options.add_argument('--incognito')
firefox_options.set_preference('intl.accept_languages', 'en-US, en')
driver = webdriver.Firefox(service=FirefoxService(
                    GeckoDriverManager().install()), options=firefox_options)

[WDM] - Driver [C:\Users\HP\.wdm\drivers\geckodriver\win64\v0.34.0\geckodriver.exe] found in cache


In [525]:
driver.get(url)

In [526]:
try:
    driver.execute_script(f"document.querySelector('.ifM9O').scrollTo(0, 2000);")

    order_dropdown = driver.find_elements(By.XPATH, "//div[@jsname='XPtOyb']")[1]
    driver.execute_script("arguments[0].click();", order_dropdown)
    time.sleep(5)
    # order_item = driver.find_elements(By.XPATH, "//div[@jsname='V68bde']/div[@jsname='wQNmvb']")[1]
    # driver.execute_script("arguments[0].click();", order_item)
    # time.sleep(1)
except Exception as e:
    print("pass review order ...")
    print(e)
pass

In [527]:
soupe = BeautifulSoup(driver.page_source, 'lxml')

In [528]:
container = soupe.find('div', {'jsname':'GmP9w'})

In [529]:
cards = container.find_all('div', {'jsname':'ShBeI'})
len(cards)

20

In [530]:
# author =  cards[0].find('a', {'class':'DHIhE QB2Jof'}).text
# cards[1].find('span', {'class': 'iUtr1 CQYfx'}).text.strip().lower()[4:] 
# cards[0].find('div', {'class':'K7oBsc'}).find('span').text.replace('(Traducido por Google) ', '').replace('\xa0... Ver más', '').strip()

In [531]:
# try:
#     driver.find_element(By.XPATH, "//button[@jsname='tWT92d']").click()
# except:
#     pass
# time.sleep(10)

In [532]:
def detect(text:str) -> str:
    if text:
        lang_code = {
        'Language.ENGLISH':'en',
        'Language.GERMAN':'de',
        'Language.SPANISH':'es',
        'Language.FRENCH':'fr',
    }

    languages = [Language.ENGLISH, Language.FRENCH, Language.GERMAN, Language.SPANISH]
    detector = LanguageDetectorBuilder.from_languages(*languages).build()
    try:
        return lang_code[f"{detector.detect_language_of(text)}"]
    except:
        return ''

In [533]:
def formate_date(raw_date):
    split_date = raw_date.split(' ')
    today = datetime.now()
    language = detect(raw_date)
    if 'mes' in raw_date:
        language = 'es'
    match language:
        case "fr":

            if split_date[4] == 'jour':
                return datetime.strftime(today + timedelta(days=-1), '%d/%m/%Y')
            elif split_date[4] == 'jours':
                return datetime.strftime(today + timedelta(days=-(int(split_date[3]))), '%d/%m/%Y')
            if split_date[4] == 'semaine':
                return datetime.strftime(today + timedelta(days=-7), '%d/%m/%Y')
            elif split_date[4] == 'semaines':
                return datetime.strftime(today + timedelta(days=-7*int(split_date[3])), '%d/%m/%Y')
            elif split_date[4] == 'mois':
                if split_date[3] == 'un':
                    return datetime.strftime(today + timedelta(days=-31), '%d/%m/%Y')
                else:
                    return datetime.strftime(today + timedelta(days=-31*int(split_date[3])), '%d/%m/%Y')
            elif split_date[4] == 'an':
                return datetime.strftime(today + timedelta(days=-365), '%d/%m/%Y')
            elif split_date[4] == 'ans':
                return datetime.strftime(today + timedelta(days=-(int(split_date[3])*365)), '%d/%m/%Y')
            else:
                return datetime.strftime(today, '%d/%m/%Y')

        case"en":
            if split_date[1] == 'days':
                return datetime.strftime(today + timedelta(days=-(int(split_date[0]))), '%d/%m/%Y')
            if split_date[1] == 'week':
                return datetime.strftime(today + timedelta(days=-7), '%d/%m/%Y')
            elif split_date[1] == 'weeks':
                return datetime.strftime(today + timedelta(days=-7*int(split_date[0])), '%d/%m/%Y')
            elif split_date[1] == 'months' or split_date[1] == 'month':
                if split_date[0] == 'a':
                    return datetime.strftime(today + timedelta(days=-31), '%d/%m/%Y')
                else:
                    return datetime.strftime(today + timedelta(days=-31*int(split_date[0])), '%d/%m/%Y')
            elif split_date[1] == 'year':
                return datetime.strftime(today + timedelta(days=-365), '%d/%m/%Y')
            elif split_date[1] == 'years':
                return datetime.strftime(today + timedelta(days=-(int(split_date[0])*365)), '%d/%m/%Y')
            else:
                return datetime.strftime(today, '%d/%m/%Y')
        case "es":
            if split_date[1] == 'día':
                return datetime.strftime(today + timedelta(days=-1), '%d/%m/%Y')
            if split_date[1] == 'días':
                return datetime.strftime(today + timedelta(days=-(int(split_date[0]))), '%d/%m/%Y')
            if split_date[1] == 'semana':
                return datetime.strftime(today + timedelta(days=-7), '%d/%m/%Y')
            elif split_date[1] == 'semanas':
                return datetime.strftime(today + timedelta(days=-7*int(split_date[0])), '%d/%m/%Y')
            elif split_date[1] == 'mes' or split_date[1] == 'meses':
                if split_date[0] == 'un':
                    return datetime.strftime(today + timedelta(days=-31), '%d/%m/%Y')
                else:
                    return datetime.strftime(today + timedelta(days=-31*int(split_date[0])), '%d/%m/%Y')
            elif split_date[1] == 'año':
                return datetime.strftime(today + timedelta(days=-365), '%d/%m/%Y')
            elif split_date[1] == 'años':
                return datetime.strftime(today + timedelta(days=-(int(split_date[0])*365)), '%d/%m/%Y')
            else:
                return datetime.strftime(today, '%d/%m/%Y')

In [534]:
import random
from urllib.parse import urlparse, parse_qs


def extract() -> list:
    # print('extraction..')

    time.sleep(3)

    # try:

    try:
        accept_btn = driver.find_element(
            By.XPATH, "//span[contains(text(), 'Tout accepter') or contains(text(), 'Accept all')]")
        driver.execute_script("arguments[0].click();", accept_btn)
        time.sleep(random.randint(2, 5))
    except:
        pass

    time.sleep(5)

    # try:
    #     driver.find_element(
    #         By.XPATH, "//div[@role='listbox' and @aria-label='Menu déroulant pour filtrer les avis']").click()
    #     time.sleep(random.uniform(.5, 2.5))
    #     driver.find_element(
    #         By.XPATH, "//div[@role='option' and @data-value='2' and @aria-label='Plus récents']").click()
    #     time.sleep(random.uniform(.2, 2))
    # except:
    #     pass

    reviews = []

    btns = driver.find_elements(By.XPATH, "//a[@jsaction='KoToPc']")

    for btn in btns:
        driver.execute_script("arguments[0].click();", btn)
        time.sleep(2)

    page = driver.page_source

    soupe = BeautifulSoup(page, 'lxml')
    container = soupe.find('div', {'jsname':'GmP9w'})
    cards = container.find_all('div', {'jsname':'ShBeI'})

    for card in cards:
        # author = card.find('span', {'class': 'k5TI0'}).find('a').text.strip() if card.find('span', {'class': 'k5TI0'}) and card.find('span', {'class': 'k5TI0'}).find('a') else ""
        author = card.find('div', {'class':'Vpc5Fe'}).text.strip() if card.find('div', {'class':'Vpc5Fe'}) else ""

        comment = card.find('div', {'jscontroller': 'lgNNHf'}).text.strip() if card.find('div', {'jscontroller': 'lgNNHf'}) else ""
        
        if comment == "":
            comment = card.find('div', {'class': 'OA1nbd'}).text.strip() if card.find('div', {'class': 'OA1nbd'}) else ""
        # try:
        #     comment = card.find('div', {'class':'K7oBsc'}).find('span').text.replace('(Traducido por Google) ', '').replace('\xa0... Ver más', '').replace(" En savoir plus", "") \
        #     .replace('(Traduit par Google)', '').replace('(Traduce by Google)', '').lower().split('(original)')[-1] if card.find('div', {'class':'K7oBsc'}) else ""
        # except:
        #     comment = ""
        
        rating = card.find('div', {'class': 'dHX2k'})['aria-label'].strip() if card.find('div', {'class': 'dHX2k'}) else "0"
        if rating != "0":
            rating = rating.split(" ")[1].replace(',', '.')
            
        try:
            lang = detect(comment)
        except:
            lang = 'en'

        date_raw = card.find('span', {'class': 'y3Ibjb'}).text.lower() if card.find('span', {'class': 'y3Ibjb'}) else ""
        
        if date_raw:
            match(detect(date_raw)):
                case 'es':
                    date_raw = date_raw.lower().replace('hace ', '').replace('\xa0','').strip() if card.find('span', {'class': 'y3Ibjb'}) else ""
                case _:
                    date_raw = date_raw.lower().replace('\xa0',' ').strip() if card.find('span', {'class': 'y3Ibjb'}) else ""

        date_review = formate_date(date_raw) if date_raw else "01/01/1999"
        if date_review:
            if (author or comment or rating != "0") and datetime.strptime(date_review, '%d/%m/%Y') > datetime.now() - timedelta(days=365):

                reviews.append({
                    'rating': rating,
                    'author': author,
                    'date_review': date_review,
                    'comment': comment,
                    'language': lang,
                    'source': urlparse(driver.current_url).netloc.split('.')[1],
                    'date_visit': date_review,
                    'novisitday': "1"
                })

                # print({
                #     'rating': rating,
                #     'author': author,
                #     'date_review': date_review,
                #     'comment': comment,
                #     'language': lang,
                #     'source': urlparse(driver.current_url).netloc.split('.')[1],
                #     'date_visit': date_review,
                #     'novisitday': "1"
                # })

    print(reviews)
    return reviews
    # except Exception as e:
    #     print(e)
    #     pass


In [535]:
extract()

[{'rating': '5.0', 'author': 'Nicolas SILVESTRE', 'date_review': '01/04/2024', 'comment': 'Trés bon contact avec Maxime.Si nous mettons en vente la maison, nous ne manquerons pas de passer par luiCordialement,', 'language': 'fr', 'source': 'google', 'date_visit': '01/04/2024', 'novisitday': '1'}, {'rating': '5.0', 'author': 'Karin van Tongeren', 'date_review': '26/03/2024', 'comment': 'Très professionnel. Bon conseil par Maxime SCHAFFNER.Je recommande 👍', 'language': 'fr', 'source': 'google', 'date_visit': '26/03/2024', 'novisitday': '1'}, {'rating': '5.0', 'author': 'Coralie Portier', 'date_review': '26/03/2024', 'comment': "Je tiens à féliciter votre agence pour le suivi parfait au sujet de l'achat de mon appartement Et je tiens tout particulièrement à remercier Mr Gauthier Anduran et toute votre équipe pour le soutient que vous m'avez offert, pour chaque démarche à faire du début à la fin. Mes demandes et mes préoccupations ont été largement prises en compte par Gauthier, avec une g

[{'rating': '5.0',
  'author': 'Nicolas SILVESTRE',
  'date_review': '01/04/2024',
  'comment': 'Trés bon contact avec Maxime.Si nous mettons en vente la maison, nous ne manquerons pas de passer par luiCordialement,',
  'language': 'fr',
  'source': 'google',
  'date_visit': '01/04/2024',
  'novisitday': '1'},
 {'rating': '5.0',
  'author': 'Karin van Tongeren',
  'date_review': '26/03/2024',
  'comment': 'Très professionnel. Bon conseil par Maxime SCHAFFNER.Je recommande 👍',
  'language': 'fr',
  'source': 'google',
  'date_visit': '26/03/2024',
  'novisitday': '1'},
 {'rating': '5.0',
  'author': 'Coralie Portier',
  'date_review': '26/03/2024',
  'comment': "Je tiens à féliciter votre agence pour le suivi parfait au sujet de l'achat de mon appartement Et je tiens tout particulièrement à remercier Mr Gauthier Anduran et toute votre équipe pour le soutient que vous m'avez offert, pour chaque démarche à faire du début à la fin. Mes demandes et mes préoccupations ont été largement prise

In [536]:
all_data = []

In [537]:
def save_data(data_source:list) -> None:
    global all_data
    new_data = []
    df = pd.concat([pd.DataFrame(all_data), pd.DataFrame(data_source)])
    df.drop_duplicates(subset=['rating','author','date_review','comment','language','source','date_visit','novisitday'], inplace=True)
    for i in range(len(df)):
        new_data.append(df.iloc[i].to_dict())
    all_data = new_data

In [538]:
def load_reviews():
    def get_last_review_date():
        page = driver.page_source
        soupe = BeautifulSoup(page, 'lxml')
        last_review_cards = soupe.find_all('div', {'jsname': "Pa5DKe"})[-1]
        date_raw = last_review_cards.find('span', {'class': 'iUtr1'}).text.strip(
        ) if last_review_cards.find('span', {'class': 'iUtr1'}) else ""
        comment = last_review_cards.find('div', {'class': 'K7oBsc'}).find('span').text.strip()\
        .replace(" En savoir plus", "") if last_review_cards.find('div', {'class': 'K7oBsc'}) else ""
        try:
            lang = detect(comment)
        except:
            lang = 'en'

        return formate_date(date_raw, lang) if date_raw else "01/01/1999"

    # try:
    #     driver.find_element(
    #         By.XPATH, "//div[@aria-label='Avis' and @aria-controls='reviews' and @role='tab']").click()
    #     time.sleep(2)
    # except:
    #     pass

    # results = int(''.join([x for x in driver.find_element(
    #     By.CSS_SELECTOR, '#reviews > c-wiz > c-wiz > div > div > div > div > div.ChBWlb.TjtFVc > div.pDLIp > div > div.zhMoVd.nNUNpc > div.UkIqCb > div > span').text if x.isdigit()]))

    input("Entrer la touche entrée pour continuer ...")
    
    index = 0

    data = extract()
    save_data(data)

    print(len(all_data))

In [539]:
# load_reviews()

In [540]:
(datetime.now() - timedelta(days=365)).strftime('%d-%m-%Y')

'03-04-2023'